In [1]:
import src
import pvlib
import numpy as np
import pandas as pd

# 1. Step 1. Upload input dataset

In [2]:
df = pd.read_csv(filepath_or_buffer=f'./validations/data/bogota-5.csv', sep=',', decimal='.', index_col='timestamp', parse_dates=True)
df.head()

,ghi_wm2,poa_wm2,tmod_c,tamb_c
timestamp,,,,
2020-01-01 00:00:00,0.0,1.022,14.682,27.484
2020-01-01 00:05:00,0.0,1.018,14.432,27.360
2020-01-01 00:10:00,0.0,1.014,13.980,26.912
2020-01-01 00:15:00,0.0,1.008,13.288,26.331
2020-01-01 00:20:00,0.0,1.000,12.772,25.794


## 1.1. Clear-sky index

If the clear-sky index ($k_c$) is not in the input dataset, it can be estimated with the `pvlib` library.

In [3]:
# Location
latitude = 4.6024
longitude = -74.0674
tz = 'America/Bogota'
altitude = 2624 # [m]

location = pvlib.location.Location(latitude, longitude, tz, altitude)

In [4]:
# Clear-sky irradiance (Hcs)
RESOLUTION = int(pd.Series(df.index.values).diff().median().total_seconds()/60)

# Clear-sky irradiance (Hcs)
hcs = location.get_clearsky(times=pd.date_range(start=df.index[0],end=df.index[-1], freq=f'{RESOLUTION}min', tz=tz),
                            model='ineichen')

hcs.head()

,ghi,dni,dhi
2020-01-01 00:00:00-05:00,0.0,0.0,0.0
2020-01-01 00:05:00-05:00,0.0,0.0,0.0
2020-01-01 00:10:00-05:00,0.0,0.0,0.0
2020-01-01 00:15:00-05:00,0.0,0.0,0.0
2020-01-01 00:20:00-05:00,0.0,0.0,0.0


In [5]:
# Append clear-sky irradiance to main dataframe
df['hcs_wm2'] = hcs['ghi'].values

# Clear-sky index (kc) to main dataframe
df['kc'] = df['poa_wm2'].values / df['hcs_wm2'].values

# NaN
df['kc'] = df['kc'].fillna(1)

# Replace kc > 1
df.loc[df['kc'] > 1, 'kc'] = 1

df.head()

,ghi_wm2,poa_wm2,tmod_c,tamb_c,hcs_wm2,kc
timestamp,,,,,,
2020-01-01 00:00:00,0.0,1.022,14.682,27.484,0.0,1.0
2020-01-01 00:05:00,0.0,1.018,14.432,27.360,0.0,1.0
2020-01-01 00:10:00,0.0,1.014,13.980,26.912,0.0,1.0
2020-01-01 00:15:00,0.0,1.008,13.288,26.331,0.0,1.0
2020-01-01 00:20:00,0.0,1.000,12.772,25.794,0.0,1.0


# 2. Step 2. Define parameters

In [6]:
DF = df                 # Input dataset. Must be a pd.Dataframe.
COL = 'poa_wm2'         # Column of the pd.DataFrame of the input data.
YEAR = 2023             # Year of study. Must exist in the input dataset.
MONTH = 1               # Month of study (1 to 12). Must exist in the input dataset.
SC = 'sc5'              # Sky condition of study. Must be 'sc1', 'sc2', 'sc3', 'sc4' or 'sc5'.
METHOD = 'stochastic'   # Method to generate the synthetic data. Must be 'stochastic' or 'bootstrap'.
IC = 0.95               # Confidence interval. Must be a value between 0 and 1.
RUNS = 5                # Quantity of one-day synthetic sequences to generate.

# 3. Step 3. Execute the synthetic data generation method

## 3.1. Unique date execution

Use the following code to run the synthetic build method once for the parameters defined above.

In [7]:
synthetic = src.methods.sequential(data=DF,
                                   irradiance_column=COL,
                                   year=YEAR,
                                   month=MONTH,
                                   sky_condition=SC,
                                   method=METHOD,
                                   IC=IC,
                                   resolution=RESOLUTION,
                                   runs=RUNS)

synthetic.head()

synt1     synt2     synt3     synt4     synt5
hour minute                                                  
0    0       0.928012  0.932419  0.932505  0.931660  0.925505
     5       0.928000  0.931131  0.928000  0.928534  0.925505
     10      0.926129  0.930195  0.926602  0.929470  0.926556
     15      0.930000  0.930000  0.930000  0.930000  0.927665
     20      0.931847  0.930334  0.932000  0.930431  0.930946

## 3.2. Multiple execution

If you want to generate the synthetic data for the entire input data set, run the following code. It will generate a one-day `RUNS` solar irradiance sequence for all sky conditions for each month and year in the input data set.

In [8]:
SYNTHETIC = {'stochastic': {}, 'bootstrap': {}}

LOOP_YEARS = DF.index.year.unique().values
LOOP_MONTHS = DF.index.month.unique().values
LOOP_SC = ['sc1', 'sc2', 'sc3', 'sc4', 'sc5']

MONTHS = {'1': 'Jan', '2': 'Feb', '3': 'Mar', '4': 'Apr', '5': 'May', '6': 'Jun',
          '7': 'Jul', '8': 'Aug', '9': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}

for i in LOOP_YEARS:
    for j in LOOP_MONTHS:
        for k in LOOP_SC:
            # Stochastic
            SYNTHETIC['stochastic'][f'{MONTHS[str(j)]}{i}-{k}'] = src.methods.sequential(data=DF,
                                                                                         irradiance_column=COL,
                                                                                         year=YEAR,
                                                                                         month=j,
                                                                                         sky_condition=k,
                                                                                         method='stochastic',
                                                                                         IC=IC,
                                                                                         resolution=RESOLUTION,
                                                                                         runs=RUNS)
            # Bootstrap
            SYNTHETIC['bootstrap'][f'{MONTHS[str(j)]}{i}-{k}'] = src.methods.sequential(data=DF,
                                                                                        irradiance_column=COL,
                                                                                        year=YEAR,
                                                                                        month=j,
                                                                                        sky_condition=k,
                                                                                        method='bootstrap',
                                                                                        IC=IC,
                                                                                        resolution=RESOLUTION,
                                                                                        runs=RUNS)

SYNTHETIC

{'stochastic': {'Jan2020-sc1':                 synt1     synt2     synt3     synt4     synt5
  hour minute                                                  
  0    0       1.115091  1.214833  0.073756  0.291436  0.498784
       5       0.909135  0.910000  0.000000  0.000000  0.908232
       10      0.910000  0.910000  0.000000  0.000000  0.910000
       15      0.910000  0.910000  0.000000  0.000000  0.910000
       20      0.911644  0.911955  0.000000  0.000000  0.911210
  ...               ...       ...       ...       ...       ...
  23   35      0.912427  0.952000  0.911276  0.916497  0.952000
       40      0.914000  0.952000  0.916000  0.914881  0.952000
       45      0.914958  0.956000  0.910000  0.911241  0.956000
       50      0.910000  0.952000  0.910000  0.910000  0.952000
       55      0.909258  0.948000  0.910446  0.912000  0.948000
  
  [288 rows x 5 columns],
  'Jan2020-sc2':                 synt1     synt2     synt3     synt4     synt5
  hour minute                  